# All you need in Pandas Agg--GroupBy--Pivot_Table--Crosstab

## Содержание:
* [agg](#agg)
* [groupby](#groupby)
* [pivot_table](#pivot_table)
* [crosstab](#crosstab)

In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
del data['Unnamed: 0']
data.columns =['age', 'sex', 'job', 'housing', 'saving_accounts', 'checking_account',
       'credit_amount', 'duration', 'purpose', 'risk']
data['credit_amount'] = data['credit_amount'].astype(float)
data['risk'] = data['risk'].replace(['good'], 0)
data['risk'] = data['risk'].replace(['bad'], 1)
data['risk'] = data['risk'].astype(int)
data['saving_accounts'] = data['saving_accounts'].fillna('unknown')
data['checking_account'] = data['checking_account'].fillna('unknown')
def categorize_age(age):
        if 0 <= age < 18:
            return 'дети'
        elif 18 <= age <= 44:
            return 'молодые'
        elif 45 <= age <= 59:
            return 'средний возраст'
        elif 60 <= age <= 74:
            return 'пожилой возраст'
        elif age >= 75:
            return 'долгожители'
        
data['age_category'] = data['age'].apply(categorize_age)
def categorize_amount(amount):
        if 0 <= amount < 500:
            return 'мало_просят'
        elif 500 <= amount <= 5000:
            return 'нужны_деньги'
        elif 5000 <= amount <= 10000:
            return 'нужно_много'
        elif amount > 10000:
            return 'олигархи'
        
data['amount_category'] = data['credit_amount'].apply(categorize_amount)

In [4]:
data.head()

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,risk,age_category,amount_category
0,67,male,2,own,unknown,little,1169.0,6,radio/TV,0,пожилой возраст,нужны_деньги
1,22,female,2,own,little,moderate,5951.0,48,radio/TV,1,молодые,нужно_много
2,49,male,1,own,little,unknown,2096.0,12,education,0,средний возраст,нужны_деньги
3,45,male,2,free,little,little,7882.0,42,furniture/equipment,0,средний возраст,нужно_много
4,53,male,2,free,little,little,4870.0,24,car,1,средний возраст,нужны_деньги


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

**Агрегация**, или агрегирование (лат. aggregatio «присоединение») — процесс объединения элементов в одну систему.

Агрегация -- там, где нужно что-то объединить по признаку или признакам.

Показать уникальные значения или максимум, минимум -- тоже агрегация.

In [5]:
data['housing'].unique()

array(['own', 'free', 'rent'], dtype=object)

## agg <a class="anchor" id=agg></a>

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html

Можно указать одну или несколько функций, агрегировать по столбцам (axis=0 по умолчанию) или по строкам (axis=1).

In [6]:
data['age'].agg('min')

19

In [7]:
data['age'].agg(['min', 'max', 'sum'])

min       19
max       75
sum    35546
Name: age, dtype: int64

In [8]:
data.agg({'age': 'max', 'credit_amount': 'median'})

age                75.0
credit_amount    2319.5
dtype: float64

Сагрегировать и сразу **переобозвать** строки:

In [9]:
data.agg(max_age=('age', 'max'), median_credit_amount=('credit_amount', 'min'))

,age,credit_amount
max_age,75.0,NaN
median_credit_amount,NaN,250.0


Можно применить не стандартныую, а **лямбда-функцию** //в нашем случае зачем-то возвели возраст в квадрат и вычли 100//

In [10]:
data.agg({'age': lambda x: x**2 - 100, 'risk': ['sum']})

,age,risk
,<lambda>,risk
0,4389.0,NaN
1,384.0,NaN
2,2301.0,NaN
3,1925.0,NaN
4,2709.0,NaN
...,...,...
996,1500.0,NaN
997,1344.0,NaN
998,429.0,NaN


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

## groupby <a class="anchor" id=groupby></a>

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html

Если не указать, какую функцию применить, то получится просто объект GroupBy:

In [11]:
data.groupby('purpose')

Вот такие группы есть;

к каждой можно обратиться по имени словаря business, domestic appliances и пр.

In [12]:
data.groupby('purpose').groups

{'business': [11, 17, 29, 30, 33, 60, 62, 63, 73, 82, 85, 95, 97, 109, 110, 145, 154, 169, 180, 181, 190, 191, 192, 202, 208, 212, 213, 216, 223, 237, 243, 245, 266, 290, 294, 343, 348, 365, 375, 384, 388, 389, 401, 402, 412, 430, 439, 464, 468, 478, 491, 512, 564, 573, 592, 597, 599, 606, 615, 620, 622, 645, 658, 667, 670, 671, 674, 684, 703, 704, 728, 729, 739, 745, 752, 785, 790, 810, 827, 829, 830, 832, 843, 861, 868, 872, 886, 887, 890, 913, 914, 950, 951, 973, 977, 981, 986], 'car': [4, 7, 9, 10, 13, 14, 18, 20, 22, 23, 32, 43, 44, 45, 47, 48, 51, 53, 54, 55, 58, 67, 70, 75, 78, 88, 91, 94, 99, 100, 104, 106, 107, 112, 113, 119, 121, 123, 125, 128, 129, 130, 136, 140, 146, 147, 153, 158, 161, 163, 164, 170, 175, 179, 182, 184, 186, 187, 198, 201, 204, 205, 209, 219, 224, 231, 236, 240, 242, 247, 250, 252, 258, 262, 264, 268, 270, 272, 280, 284, 285, 286, 291, 292, 293, 296, 297, 300, 302, 303, 304, 306, 309, 313, 314, 320, 322, 324, 325, 327, ...], 'domestic appliances': [38, 162

Более содержательно, если указать **функцию для сгруппированных данных**;

рассчитана сумма кредитов по целям кредитов; можно было НЕ указать столбцец --> суммы были бы рассчитаны по всем столбцам.

Использовать **dropna=False**, чтобы пропуски тоже учитывались.

In [13]:
data.groupby('purpose', dropna=False).agg({'credit_amount': 'sum'})

,credit_amount
purpose,
business,403330.0
car,1269881.0
domestic appliances,17976.0
education,169873.0
furniture/equipment,555125.0
radio/TV,696543.0
repairs,60018.0
vacation/others,98512.0


Вариант записи того же самого:

In [14]:
data.groupby('purpose')['credit_amount'].sum()

purpose
business                403330.0
car                    1269881.0
domestic appliances      17976.0
education               169873.0
furniture/equipment     555125.0
radio/TV                696543.0
repairs                  60018.0
vacation/others          98512.0
Name: credit_amount, dtype: float64

In [15]:
data.groupby('purpose').agg({'credit_amount': ['min', 'max', 'mean']})

credit_amount                      
                              min      max         mean
purpose                                                
business                    609.0  15945.0  4158.041237
car                         250.0  14896.0  3768.192878
domestic appliances         343.0   3990.0  1498.000000
education                   339.0  12612.0  2879.203390
furniture/equipment         428.0  14179.0  3066.988950
radio/TV                    338.0  15653.0  2487.653571
repairs                     454.0  11998.0  2728.090909
vacation/others            1164.0  18424.0  8209.333333

**Переобозвать** столбцы на выходе:

In [16]:
data.groupby('purpose').agg(minimalka=('credit_amount', 'min'), maximalka=('credit_amount', 'max'))

,minimalka,maximalka
purpose,,
business,609.0,15945.0
car,250.0,14896.0
domestic appliances,343.0,3990.0
education,339.0,12612.0
furniture/equipment,428.0,14179.0
radio/TV,338.0,15653.0
repairs,454.0,11998.0
vacation/others,1164.0,18424.0


По умолчанию **сортировка** -- по признаку группировки. `purpose` отсортирован по алфавиту.

Если это не нравится, то можно указать `sort=False`

In [17]:
data.groupby('purpose', sort=False).agg({'credit_amount': ['min', 'max', 'mean']})

credit_amount                      
                              min      max         mean
purpose                                                
radio/TV                    338.0  15653.0  2487.653571
education                   339.0  12612.0  2879.203390
furniture/equipment         428.0  14179.0  3066.988950
car                         250.0  14896.0  3768.192878
business                    609.0  15945.0  4158.041237
domestic appliances         343.0   3990.0  1498.000000
repairs                     454.0  11998.0  2728.090909
vacation/others            1164.0  18424.0  8209.333333

Можно группируемый признак **выкинуть из индексов** с помощью `as_index=False`:

In [18]:
data.groupby('purpose', as_index=False).agg({'credit_amount': ['min', 'max', 'mean']})

purpose credit_amount                      
                                 min      max         mean
0             business         609.0  15945.0  4158.041237
1                  car         250.0  14896.0  3768.192878
2  domestic appliances         343.0   3990.0  1498.000000
3            education         339.0  12612.0  2879.203390
4  furniture/equipment         428.0  14179.0  3066.988950
5             radio/TV         338.0  15653.0  2487.653571
6              repairs         454.0  11998.0  2728.090909
7      vacation/others        1164.0  18424.0  8209.333333

Группировать по **нескольким признакам**:

In [19]:
data.groupby(['sex', 'purpose']).agg({'risk': 'sum'}).reset_index()

,sex,purpose,risk
0,female,business,7
1,female,car,40
2,female,domestic appliances,2
3,female,education,9
4,female,furniture/equipment,28
5,female,radio/TV,19
6,female,repairs,2
7,female,vacation/others,2
8,male,business,27
9,male,car,66


Можно получить **аналог сводной таблицы** через `unstack`:

In [20]:
data.groupby(['sex', 'purpose']).agg({'risk': 'sum'}).unstack('sex')

risk     
sex                 female male
purpose                        
business                 7   27
car                     40   66
domestic appliances      2    2
education                9   14
furniture/equipment     28   30
radio/TV                19   43
repairs                  2    6
vacation/others          2    3

**Если в данных есть пропуски.**

Учесть их через dropna=False. //В примере нет пропусков.//

In [21]:
data.groupby('sex', dropna=False).agg({'risk': 'sum'}).reset_index()

,sex,risk
0,female,109
1,male,191


**Раскрасить результаты** через `style.background_gradient`:

In [22]:
data.groupby('purpose', as_index=False).agg({'credit_amount': ['min', 'max', 'mean']}).style.background_gradient(cmap='pink')

Если нужно посчитать что-то, чему **нет метода в pandas**:

In [23]:
# например, как рассчитать величину размаха max - min:
def max_min(data):
    max_value = data['credit_amount'].max()
    min_value = data['credit_amount'].min()
    
    return max_value-min_value

data.groupby('sex').apply(max_min)

sex
female    18174.0
male      15669.0
dtype: float64

После группировок для проверки можно посчитать **контрольную сумму**:

In [24]:
data_gr_sex = data.groupby('sex', dropna=False).agg({'risk': ['count', 'sum']}).reset_index()
data_gr_sex

sex  risk     
          count  sum
0  female   310  109
1    male   690  191

In [25]:
data_gr_sex['risk', 'count'].sum()

1000

In [26]:
data.shape

(1000, 12)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

## pivot_table <a class="anchor" id=pivot_table></a>

https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html

**По умолчанию** посчитал среднее.

In [27]:
data.pivot_table(index = 'age_category')

,age,credit_amount,duration,job,risk
age_category,,,,,
долгожители,75.000000,3994.500000,15.000000,3.000000,0.000000
молодые,30.908636,3272.161452,21.121402,1.912390,0.312891
пожилой возраст,64.510204,3289.204082,18.979592,1.816327,0.265306
средний возраст,50.260000,3250.940000,20.446667,1.873333,0.246667


Можно сделать **мультииндекс**: зависимость от ___ И __.

In [28]:
data.pivot_table(index=['age_category', 'amount_category'])

age  credit_amount   duration  \
age_category    amount_category                                        
долгожители     нужно_много      75.000000    6615.000000  24.000000   
                нужны_деньги     75.000000    1374.000000   6.000000   
молодые         мало_просят      28.846154     398.000000   8.538462   
                нужно_много      31.815789    6935.052632  33.403509   
                нужны_деньги     30.714063    2222.500000  18.259375   
                олигархи         32.406250   12383.968750  39.718750   
пожилой возраст нужно_много      66.333333    6683.000000  24.500000   
                нужны_деньги     64.300000    1940.975000  16.425000   
                олигархи         63.666667   14478.000000  42.000000   
средний возраст мало_просят      51.200000     370.400000   8.400000   
                нужно_много      50.296296    7101.296296  30.740741   
                нужны_деньги     50.176991    2018.584071  17.566372   
                олигархи         51.000000   13190.800000  42.000000   

                                      job      risk  
age_category    amount_category                      
долгожители     нужно_много      3.000000  0.000000  
                нужны_деньги     3.000000  0.000000  
молодые         мало_просят      1.615385  0.230769  
                нужно_много      2.131579  0.324561  
                нужны_деньги     1.857813  0.298438  
                олигархи         2.343750  0.593750  
пожилой возраст нужно_много      2.500000  0.500000  
                нужны_деньги     1.625000  0.200000  
                олигархи         3.000000  0.666667  
средний возраст мало_просят      1.400000  0.000000  
                нужно_много      2.333333  0.518519  
                нужны_деньги     1.761062  0.176991  
                олигархи         2.400000  0.600000

Выкинули лишние колонки и смотрим только риск и указали, что считаем суммы:

In [29]:
data.pivot_table(index='age_category',
                 values='risk',
                 aggfunc='sum')

,risk
age_category,
долгожители,0
молодые,250
пожилой возраст,13
средний возраст,37


Теперь **несколько функций**:

In [30]:
data.pivot_table(index='age_category',
                 values='risk',
                 aggfunc=['count', 'sum'])

,count,sum
,risk,risk
age_category,,
долгожители,2,0
молодые,799,250
пожилой возраст,49,13
средний возраст,150,37


Введём **колонки**:

In [31]:
data.pivot_table(index='age_category',
                 columns='sex',
                 values='risk',
                 aggfunc=['count', 'sum'],
                 fill_value=0)

count         sum     
sex             female male female male
age_category                           
долгожители          1    1      0    0
молодые            261  538    100  150
пожилой возраст     11   38      2   11
средний возраст     37  113      7   30

А теперь **несколько колонок**:

In [32]:
data.pivot_table(index='age_category',
                 columns=['sex', 'job'],
                 values='risk',
                 aggfunc=['count', 'sum'],
                 fill_value=0)

count                                    sum              \
sex             female              male               female               
job                  0   1    2   3    0    1    2   3      0   1   2   3   
age_category                                                                
долгожители          0   0    0   1    0    0    0   1      0   0   0   0   
молодые              9  48  177  27    6  105  341  86      4  20  63  13   
пожилой возраст      1   1    6   3    4    9   17   8      0   0   0   2   
средний возраст      2  15   14   6    0   22   75  16      1   2   3   1   

                                  
sex             male              
job                0   1   2   3  
age_category                      
долгожители        0   0   0   0  
молодые            2  27  95  26  
пожилой возраст    0   3   5   3  
средний возраст    0   4  20   6

Добавление **итогов** через `margins`:

In [33]:
data.pivot_table(index=['age_category', 'sex'],
                 columns=['job'],
                 values='risk',
                 aggfunc=['sum'],
                 fill_value=0,
                 margins=True,
                 margins_name='ToTal')

sum                   
job                      0   1    2   3 ToTal
age_category    sex                          
долгожители     female   0   0    0   0     0
                male     0   0    0   0     0
молодые         female   4  20   63  13   100
                male     2  27   95  26   150
пожилой возраст female   0   0    0   2     2
                male     0   3    5   3    11
средний возраст female   1   2    3   1     7
                male     0   4   20   6    30
ToTal                    7  56  186  51   300

**Разные функции** для разных данных -- словарь для функций:

In [34]:
data.pivot_table(index=['purpose'],
                 columns=['sex'],
                 values=['risk', 'duration'],
                 aggfunc={'risk': 'sum', 'duration': 'mean'},
                 fill_value=0,
                 margins=True,
                 dropna=True,
                 margins_name='ToTal')

duration                         risk           
sex                     female       male      ToTal female male ToTal
purpose                                                               
business             27.000000  26.923077  26.938144      7   27    34
car                  20.095745  21.090535  20.813056     40   66   106
domestic appliances  15.500000  18.166667  16.833333      2    2     4
education            13.541667  23.885714  19.677966      9   14    23
furniture/equipment  17.891892  20.261682  19.292818     28   30    58
radio/TV             19.635294  20.225641  20.046429     19   43    62
repairs              16.200000  19.941176  19.090909      2    6     8
vacation/others      44.000000  28.444444  32.333333      2    3     5
ToTal                19.438710  21.560870  20.903000    109  191   300

В функции можно засунуть и **лямбда-функцию**:

In [35]:
data.pivot_table(index=['purpose'],
                 columns=['sex'],
                 values=['duration'],
                 aggfunc=lambda x: list(x))

duration  \
sex                                                             female   
purpose                                                                  
business             [48, 42, 18, 12, 12, 48, 9, 12, 33, 27, 15, 21...   
car                  [12, 15, 24, 48, 11, 6, 18, 36, 36, 24, 12, 48...   
domestic appliances                             [15, 9, 6, 12, 36, 15]   
education            [12, 15, 9, 6, 12, 12, 12, 18, 24, 18, 9, 6, 1...   
furniture/equipment  [12, 36, 18, 24, 10, 33, 18, 21, 12, 18, 11, 2...   
radio/TV             [48, 12, 24, 12, 12, 36, 24, 12, 36, 24, 12, 6...   
repairs                                           [18, 12, 15, 18, 18]   
vacation/others                                           [24, 60, 48]   

                                                                        
sex                                                               male  
purpose                                                                 
business             [30, 60, 18, 12, 9, 36, 48, 54, 18, 14, 6, 48,...  
car                  [24, 36, 30, 24, 9, 10, 12, 18, 30, 11, 27, 18...  
domestic appliances                            [10, 15, 12, 18, 48, 6]  
education            [12, 36, 48, 36, 36, 36, 36, 12, 21, 9, 12, 12...  
furniture/equipment  [42, 24, 10, 6, 24, 24, 30, 24, 9, 12, 15, 18,...  
radio/TV             [6, 12, 24, 24, 6, 6, 7, 45, 18, 9, 30, 12, 36...  
repairs              [18, 27, 12, 12, 9, 36, 30, 24, 6, 42, 15, 24,...  
vacation/others                    [8, 24, 48, 48, 24, 20, 24, 24, 36]

**Сортировать** как по любой колонке, так и по индексу через `sort=True`:

In [36]:
data.pivot_table(index=['purpose'],
                 columns=['sex'],
                 values=['risk', 'duration'],
                 aggfunc={'risk': 'sum', 'duration': 'mean'},
                 #sort=False
                )\
.sort_values(by = ('duration', 'male'), ascending = False)

duration              risk     
sex                     female       male female male
purpose                                              
vacation/others      44.000000  28.444444      2    3
business             27.000000  26.923077      7   27
education            13.541667  23.885714      9   14
car                  20.095745  21.090535     40   66
furniture/equipment  17.891892  20.261682     28   30
radio/TV             19.635294  20.225641     19   43
repairs              16.200000  19.941176      2    6
domestic appliances  15.500000  18.166667      2    2

**Избавиться от двухэтажного названия колонок.**

In [37]:
a = data.pivot_table(index=['purpose'],
                 columns=['sex'],
                 values=['risk', 'duration'],
                 aggfunc={'risk': 'sum', 'duration': 'mean'})

a.columns = ['_'.join(col).strip() for col in a.columns.values]
a

,duration_female,duration_male,risk_female,risk_male
purpose,,,,
business,27.000000,26.923077,7,27
car,20.095745,21.090535,40,66
domestic appliances,15.500000,18.166667,2,2
education,13.541667,23.885714,9,14
furniture/equipment,17.891892,20.261682,28,30
radio/TV,19.635294,20.225641,19,43
repairs,16.200000,19.941176,2,6
vacation/others,44.000000,28.444444,2,3


**Раскрасить результаты** через `style.background_gradient`:

In [44]:
data.pivot_table(index=['purpose'],
                 columns=['sex'],
                 values=['risk', 'duration'],
                 aggfunc={'risk': 'sum', 'duration': 'mean'})\
.style.background_gradient(cmap='pink')

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

## crosstab <a class="anchor" id=crosstab></a>

Вычислите простую перекрестную таблицу двух (или более) факторов.

По умолчанию вычисляет таблицу **частот факторов**, если не переданы массив значений и функция агрегирования.

https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html

In [39]:
pd.crosstab(data['sex'], data['job'])

job,0,1,2,3
sex,,,,
female,12,64,197,37
male,10,136,433,111


Как если бы сделали сводную, где посчитали количество значений:

In [40]:
data.pivot_table(index='sex', columns='job', values='duration', aggfunc='count')

job,0,1,2,3
sex,,,,
female,12,64,197,37
male,10,136,433,111


Можно указать **итоги**:

In [41]:
pd.crosstab(data['sex'], data['job'], margins=True, margins_name='ALL')

job,0,1,2,3,ALL
sex,,,,,
female,12,64,197,37,310
male,10,136,433,111,690
ALL,22,200,630,148,1000


Можно превратить в полный аналог pivot_table:

In [42]:
pd.crosstab(data['sex'], data['job'],
            values=data['duration'],
            aggfunc='mean')

job,0,1,2,3
sex,,,,
female,16.75,17.625000,19.218274,24.621622
male,18.10,16.022059,22.408776,25.351351


**normalize**

Разделить все значения на сумму значений.

-- 'all' или True -- нормализуется по всем значениям;

-- index -- для каждой строки;

-- columns -- по каждому столбцу.

In [43]:
pd.crosstab(data['sex'], data['job'],
            values=data['credit_amount'],
            aggfunc='sum',
           normalize='index',
           margins=True)

job,0,1,2,3
sex,,,,
female,0.027786,0.168800,0.584066,0.219349
male,0.014965,0.134971,0.594186,0.255877
All,0.018462,0.144197,0.591426,0.245915
